# Week 7 Lecture 1 - Le Poisson

McElreath's lecture for today: https://www.youtube.com/watch?v=YrwL6t0kW2I&list=PLDcUM9US4XdMROZ57-OIRtIK0aOynbgZN&index=11

McElreath's lectures for the whole book are available here: https://github.com/rmcelreath/stat_rethinking_2022

An R/Stan repo of code is available here: https://vincentarelbundock.github.io/rethinking2/

An excellent port to Python/PyMC Code is available here: https://github.com/dustinstansbury/statistical-rethinking-2023

You are encouraged to work through both of these versions to re-enforce what we're doing in class.

In [ ]:
# Import python packages
%matplotlib inline
import pandas as pd
import numpy as np
import seaborn as sns
import scipy as sp 
import random as rd
import pdb
import pymc as pm
import dataframe_image as dfi
import networkx as nx
import patsy
import arviz as az
from matplotlib import pyplot as plt


# Helper functions
def stdize(x):
    return (x-np.mean(x))/np.std(x)


def indexall(L):
    poo = []
    for p in L:
        if not p in poo:
            poo.append(p)
    Ix = np.array([poo.index(p) for p in L])
    return poo,Ix

def indexall_(L):
    Il, Ll = pd.factorize(L, sort=True)
    return Ll, Il

## Oceanic tool complexity

A fascinating set of data from [Michelle Kline](https://www.michelleakline.com/), whereby she collected information about the size and complexity of tools among polyensian and melanesian societies. The idea is that larger, more connected populations should have more complex sets of tools. So let's take a look and see:

In [ ]:
# Grab tool data
kdata = pd.read_csv('Kline.csv', sep=';')
dfi.export(kdata, 'kline.png')
kdata

And this is the entire dataset - this should give us pause about sample sizes and why McElreath is so focused on regularization, but also give us hope that we can make inferences from data in a Bayesian context by thinking carefully about what we're doing. 

Here we want to know: does the number of tools increase with log(population) size and the contact rate among islands? To do this, we can build a Poisson model for tool counts:

$$
\begin{align}
T_i \sim & Poisson(\lambda_i) \\
log(\lambda_i) = & \beta_c + \beta_p log(P)
\end{align}
$$

where there are distinct interepts $\beta_c$ and population size effects $\beta_b$ for high and low contact islands. Given how small this dataset is, priors become very important. Let's simulate first some intercepts to see what the Poisson and log link do to our numbers.

In [ ]:
plt.figure(figsize=(6, 4))
# Plot priors from N(0, 10)
ax = az.plot_kde(np.random.lognormal(0,10,1000), label="Bc ~ LN(0, 10)")
ax.set_xlabel("mean number of tools")
ax.set_ylabel("Density")
ax.set_title("exp(Bc)")
plt.savefig('ln10.jpg',dpi=300);

Wow, those numbers get big fast - how big? Well the mean value is

In [ ]:
np.mean(np.exp(np.random.normal(0,10,10000)))

a boatload. Picking something sensible can be hard. If we eyeball the total tools column stuff is in the 10's (between 13 and 71), so let's try some alternatives:

In [ ]:
np.mean(np.exp(np.random.normal(2,2,10000)))

Not bad, let's see what it looks like

In [ ]:
plt.figure(figsize=(6, 4))
# Plot priors from N(2, 2)
ax = az.plot_kde(np.random.lognormal(2,2,10000), label="Bc ~ LN(2, 2)")
ax.set_xlabel("mean number of tools")
ax.set_ylabel("Density")
ax.set_title("exp(Bc)")
plt.savefig('ln2.jpg',dpi=300);

Hummm...not ideal - 1000's of tools isn't on the radar, maybe tighten up the variance a bit

In [ ]:
np.mean(np.exp(np.random.normal(2,.5,10000)))

That seems low, maybe increase the mean a bit

In [ ]:
np.mean(np.exp(np.random.normal(3,.5,10000))), np.exp(3)

Better, let's take a look

In [ ]:
plt.figure(figsize=(6, 4))
# Plot priors from N(3, .5)
ax = az.plot_kde(np.random.lognormal(3,0.5,10000), label="Bc ~ LN(3, 0.5)")
ax.set_xlabel("mean number of tools")
ax.set_ylabel("Density")
ax.set_title("exp(Bc)")
plt.savefig('ln05.jpg',dpi=300);

Ok, that seems reasonable for the intercept, but what about the slope? We'll likely use standardized log-population size, so let's see what that looks like

In [ ]:
# Number of samples
N = 100
# Intercept prior
b0 = np.random.normal(3, 0.5, N)
# log-Population size prior
b1 = np.random.normal(0, 0.5, N)
# log-population (std)
xnew = stdize(np.log(np.linspace(100,200_000,1000)))
# Plot predicted lines
[plt.plot(xnew, np.exp(b0_+b1_*xnew)) for b0_,b1_ in zip(b0,b1)]
plt.ylim(0,100)
plt.xlim(-1,1)
plt.axvline(0,c='black')
plt.xlabel('log(population) (std)')
plt.ylabel('total tools')
plt.title("Prior predictive lines")
plt.savefig('ppl.jpg',dpi=300);

Looks reasonable, so let's grab the data and run

In [ ]:
## Kline data

# Total tools - response
T = kdata.total_tools.values

# log-Population size (std)
P = stdize(np.log(kdata.population.values))
# Dummy for high-contact
C,Ic = indexall(kdata.contact.values)
C

In [ ]:
with pm.Model(coords={'Contact':C}) as Tools:
    # Data
    T_ = pm.Data('Tools', T, mutable=True)
    P_ = pm.Data('zPopulation', P, mutable=True)
    Ic_ = pm.Data('Ic', Ic, mutable=True)
    
    # Contact intercepts
    β0 = pm.Normal('Intercept', 3, 0.5, dims='Contact')
    # Contact effects of log-population size
    β1 = pm.Normal('logPop', 0, 0.2, dims='Contact')
    
    # Linear model
    λ = pm.math.exp(β0[Ic_]+β1[Ic_]*P_)

    # Likelihood
    Yi = pm.Poisson('TotalTools', λ, observed=T_)

In [ ]:
with Tools:
    trace_t = pm.sample(1000)

In [ ]:
tmp = pm.summary(trace_t)
#dfi.export(tmp.style.background_gradient(), 'df_m1.png')
dfi.export(tmp, 'df_m1.png')
tmp

So it looks like log(population) size matters for low contact places, but not for high.

To illustrate something key about parameters and overfitting in the Poisson context, let's also run an intercept-only model:

In [ ]:
with pm.Model() as Tools_null:
    # Data
    T_ = pm.Data('Tools', T, mutable=True)
    P_ = pm.Data('zPopulation', P, mutable=True)
    Ic_ = pm.Data('Ic', Ic, mutable=True)
    
    # Contact intercepts
    β0 = pm.Normal('Intercept', 3, 0.5)

    # Linear model
    λ = pm.math.exp(β0)

    # Likelihood
    Yi = pm.Poisson('TotalTools', λ, observed=T_)

In [ ]:
with Tools_null:
    trace_n = pm.sample(1000)

In [ ]:
tmp = pm.summary(trace_n)
#dfi.export(tmp.style.background_gradient(), 'df_m1.png')
dfi.export(tmp, 'df_m0.png')
tmp

In [ ]:
# Calculate tools loo
with Tools_null:
    pm.compute_log_likelihood(trace_n)
with Tools:
    pm.compute_log_likelihood(trace_t)
tools_loo = pm.loo(trace_t, pointwise=True)
tools_loo

In [ ]:
tmp = az.compare({"Tools - logPop": trace_t, "Tools - null": trace_n})
dfi.export(tmp, 'lootable.png')
tmp

We can see the log-population model does better than the null - but also that there are loo-based warnings. Where are thse coming from? We can plot the data with our model, scaling the point sizes by their Pareto-k values to see where the issues are (remember the chimps?)


In [ ]:
## Predicted values from the model using PyMC

# Number of values
N = 10
# Range of standardized population
xnew = np.linspace(min(P),max(P),N)

In [ ]:
with Tools:
    # Low contact predictions
    pm.set_data({'Ic':np.zeros(N).astype(int), 'zPopulation':xnew})
    ynew0_ = pm.sample_posterior_predictive(trace_t, var_names=["TotalTools"], predictions=True)
    # High contact predictions
    pm.set_data({'Ic':np.ones(N).astype(int), 'zPopulation':xnew})
    ynew1_ = pm.sample_posterior_predictive(trace_t, var_names=["TotalTools"], predictions=True)

In [ ]:
# Grab intervals
ynew0 = ynew0_.predictions.TotalTools
ynew1 = ynew1_.predictions.TotalTools
# Grab trendlines
ymu0 = ynew0.values.mean(1)[0]
ymu1 = ynew1.values.mean(1)[0]


In [ ]:
# Grab pointwise k values
k = tools_loo.pareto_k.values

In [ ]:
## Plot model fits with pareto k values
# Code from https://github.com/AlexAndorra
_, (ax0, ax1) = plt.subplots(1, 2, figsize=(12, 6))
az.style.use("arviz-darkgrid")

# Point size scaled to pareto k 
psize = k/k.max()*250

# = = = = = = = = = = = = = = = = Plot standardized scale
# Plot low-connected fit
az.plot_hdi(xnew, ynew0, color="b", fill_kwargs={"alpha": 0.2}, ax=ax0)
ax0.plot(xnew, ymu0, "--", color="b", alpha=0.7, label="Low contact expected")

# Plot highly-connected fit
az.plot_hdi(xnew, ynew1, color="b", fill_kwargs={"alpha": 0.2}, ax=ax0)
ax0.plot(xnew, ymu1,  color="b", alpha=0.7, label="High contact expected")

# Plot names and k values
mask = k>0.5
labels = kdata.culture.values[mask]
[ax0.text(P[mask][i]-0.2, T[mask][i]+4, f"{text}({np.round(k[mask][i], 2)})",fontsize=8,) for i, text in enumerate(labels)]

# Plot highly-connected data
indx = Ic == C.index('low')
ax0.scatter(P[indx], T[indx],s=psize[indx],facecolors="none",edgecolors="k",alpha=0.8,lw=1,label="low contact")
# Plot low-connected data
ax0.scatter(P[~indx], T[~indx],s=psize[~indx],alpha=0.8,lw=1,label="high contact")

# Make pretty
ax0.set_xlabel("log(Population) (std)")
ax0.set_ylabel("Total tools")
ax0.legend(fontsize=8, ncol=2)
ax0.set_ylim(0,130)


# = = = = = = = = = = = = = = = = Plot natural scale
# Undstandardize prediction scale
tmp = np.exp(xnew*np.log(kdata.population.values).std()+np.log(kdata.population.values).mean())
xnew2 = az.from_dict(posterior={"T": T}, constant_data={"P":tmp}).constant_data.P

# Plot low-connected fit
az.plot_hdi(xnew2, ynew0, color="b", fill_kwargs={"alpha": 0.2}, ax=ax1)
ax1.plot(xnew2.values, ymu0, "--", color="b", alpha=0.7, label="Low contact mean")

# Plot highly-connected fit
az.plot_hdi(xnew2, ynew1, color="b", fill_kwargs={"alpha": 0.2}, ax=ax1)
ax1.plot(xnew2.values, ymu1,  color="b", alpha=0.7, label="High contact mean")

# Plot low-connected data
ax1.scatter(kdata.population[indx], T[indx],s=psize[indx],facecolors="none",edgecolors="k",alpha=0.8,lw=1,label="low contact")
# Plot high-connected data
ax1.scatter(kdata.population[~indx], T[~indx],s=psize[~indx],alpha=0.8,lw=1,label="high contact")
ax1.set_xlim(0,300000)
plt.setp(ax1.get_xticklabels(), ha="right", rotation=45)
ax1.set_xlabel("Population")
ax1.set_ylabel("Total tools")
ax1.set_ylim(0,130)
plt.tight_layout()

plt.savefig('modelfits.jpg',dpi=300);

With a $N(0, 0.2)$ prior for the slopes, we can see that the high contact mean trend falls below that of Hawaii - this is a problem because low-contact Hawaii has lots of tools due to high population, so we'd expect the counterfactual high-contact Hawaii to have even more tools, but the prediction says less. This is due, in part to the fact that our standardized model isn't anchored on zero for total tools at zero total population size. We can do better than this.

*Scientific model*

Based on domain knowledge - i.e. our understanding of the system under study - it should make sense that innovation in tool development increases with population size but with diminishing returns; eventually each additional person will add less innovation. Also cultures tend to discard tools over time, replacing them with new ones. These two processes can be represented by

$$
\Delta T = \alpha P^{\beta} - \gamma T
$$

where the change in tool number per time step ($\Delta T$) is equal to some increase ($\alpha$) proportional to population size, with diminishing returns ($\beta$), and the per-unit-time rate of tool loss ($\gamma$). If we then set $\Delta T=0$, we'll get the equilibrium tool set size


$$
\hat{T} = \frac{\alpha P^{\beta}}{\gamma}.
$$

Which we can encode this into a statistical model as

$$
\begin{align}
T_i \sim & Poisson(\lambda_i) \\
\lambda_i = & \frac{\alpha P^{\beta}}{\gamma}
\end{align}
$$

Note that the link function is gone now, because we have a mechanistic model (science!).

In [ ]:
# Population size actual counts
p = kdata.population.values

In [ ]:
with pm.Model(coords={'Contact':C}) as SciTools:
    # Data
    T_ = pm.Data('Tools', T, mutable=True)
    P_ = pm.Data('Population', p, mutable=True)
    Ic_ = pm.Data('Ic', Ic, mutable=True)
    
    # Innovation increase with population
    α = pm.Normal('innovation', .5, 1, dims='Contact')
    # Diminishing returns
    β = pm.Exponential('dim_returns', 2, dims='Contact')
    # Tool loss rate
    γ = pm.Exponential('tool_loss', .5)
    
    # Scientific model
    λ = (pm.math.exp(α[Ic_])*(P_**β[Ic_]))/γ

    # Likelihood
    Yi = pm.Poisson('TotalTools', λ, observed=T_)

In [ ]:
with SciTools:
    trace_st = pm.sample(1000)

In [ ]:
az.plot_trace(trace_st, compact=True)
plt.savefig('sciencetrace.jpg',dpi=300);

Ok, things looking good here on the estimation side - how about plotting our model against our observed data

In [ ]:
## Predicted values from the model using PyMC

# Number of values
N = 10
# Range of positive standardized population
xnew = np.linspace(min(p),max(p),N, dtype='int64')

with SciTools:
    # Low contact predictions
    pm.set_data({'Ic':np.array([0]*N), 'Population':xnew})
    ynew0_ = pm.sample_posterior_predictive(trace_st)
    # High contact predictions
    pm.set_data({'Ic':np.array([1]*N), 'Population':xnew})
    ynew1_ = pm.sample_posterior_predictive(trace_st)

    
# Grab intervals
ynew0 = ynew0_.posterior_predictive.TotalTools
ynew1 = ynew1_.posterior_predictive.TotalTools
# Grab trendlines
ymu0 = ynew0.values.mean(1)[0]
ymu1 = ynew1.values.mean(1)[0]


In [ ]:
## Plot model fits with pareto k values
# Code from https://github.com/AlexAndorra
_, (ax1) = plt.subplots(1, 1, figsize=(7, 6))
az.style.use("arviz-darkgrid")

# Point size scaled to pareto k 
psize = k/k.max()*250

# = = = = = = = = = = = = = = = = Plot natural scale
# Undstandardize prediction scale
xnew2 = xnew

# Plot low-connected fit
az.plot_hdi(xnew2, ynew0, color="b", fill_kwargs={"alpha": 0.2}, ax=ax1)
ax1.plot(xnew2, ymu0, "--", color="b", alpha=0.7, label="Low contact mean")

# Plot highly-connected fit
az.plot_hdi(xnew2, ynew1, color="b", fill_kwargs={"alpha": 0.2}, ax=ax1)
ax1.plot(xnew2, ymu1,  color="b", alpha=0.7, label="High contact mean")

# Plot low-connected data
ax1.scatter(kdata.population[indx], T[indx],s=psize[indx],facecolors="none",edgecolors="k",alpha=0.8,lw=1,label="low contact")
# Plot high-connected data
ax1.scatter(kdata.population[~indx], T[~indx],s=psize[~indx],alpha=0.8,lw=1,label="high contact")

plt.setp(ax1.get_xticklabels(), ha="right", rotation=45)
ax1.set_xlabel("Population")
ax1.set_ylabel("Total tools")
ax1.set_xlim((-10000, 300000))
ax1.set_ylim(0,130)
ax1.legend(fontsize=8, ncol=2)
plt.tight_layout()
plt.savefig('sciencefits.jpg',dpi=300);

## Gamma-Poisson mixture (aka Negative Binomial)

While Poisson distributions abound in nature, the hard scaling of the mean and variance being equal to $\lambda$ means that it can fail in cases where the variance is either greater or less than we'd expect. In these cases we can turn to the [Negative Binomial](https://en.wikipedia.org/wiki/Negative_binomial_distribution), which can be thought of as (and often arises from) a mixture of Poisson distribtuions whose rates are [Gamma distributed](https://en.wikipedia.org/wiki/Gamma_distribution). This is, in my view, a much more intuitive representation than the Negative Binomial's namesake definition, which relates to the expected number of times you something fails until you see a success.

In the case of our scientific model above, we can use the Negative Binomial in place of the Poisson to allow for *overdispersion* - the rate at which the variance departs from the mean as it increases, namely:

$$
var_{NB} = \lambda+\lambda^{2}/\phi
$$

where $\phi$ conrols the rate of departure from the $mean=variance$ relationship of the Poisson. As $\phi$ increases, the NB converges toward a Poisson. 

Let's apply this model to the Oceania data and see what happens:

In [ ]:
with pm.Model(coords={'Contact':C}) as SciTools_NB:
    # Data
    T_ = pm.Data('Tools', T, mutable=True)
    P_ = pm.Data('Population', p, mutable=True)
    Ic_ = pm.Data('Ic', Ic, mutable=True)
    
    # Innovation increase with population
    α = pm.Normal('iRate', 1, 1, dims='Contact')
    # Diminishing returns
    β = pm.Exponential('dReturns', 2, dims='Contact')
    # Tool loss rate
    γ = pm.Exponential('lRate', .5)
    
    # Scientific model
    λ = (pm.math.exp(α[Ic_])*P_**β[Ic_])/γ

    # Gamma mixture parameter
    φ = pm.Gamma('phi', 0.01, 0.01)
    
    # Likelihood
    Yi = pm.NegativeBinomial('TotalTools', λ, φ, observed=T)

In [ ]:
with SciTools_NB:
    trace_stnb = pm.sample(1000)

In [ ]:
az.plot_trace(trace_stnb, compact=True);

In [ ]:
## Predicted values from the model using PyMC3

# Number of values
N = 10

with SciTools_NB:
    # Low contact predictions
    pm.set_data({'Ic':np.array([0]*N), 'Population':xnew})
    ynew0_ = pm.sample_posterior_predictive(trace_stnb)
    # High contact predictions
    pm.set_data({'Ic':np.array([1]*N), 'Population':xnew})
    ynew1_ = pm.sample_posterior_predictive(trace_stnb)
    
# Grab intervals
ynew0 = ynew0_.posterior_predictive.TotalTools
ynew1 = ynew1_.posterior_predictive.TotalTools
# Grab trendlines
ymu0 = ynew0.values.mean(1)[0]
ymu1 = ynew1.values.mean(1)[0]


In [ ]:
## Plot model fits with pareto k values
# Code from https://github.com/AlexAndorra
_, (ax1) = plt.subplots(1, 1, figsize=(7, 6))
az.style.use("arviz-darkgrid")

# Point size scaled to pareto k 
psize = k/k.max()*250

# = = = = = = = = = = = = = = = = Plot natural scale
# Undstandardize prediction scale
xnew2 = xnew

# Plot low-connected fit
az.plot_hdi(xnew2, ynew0, color="b", fill_kwargs={"alpha": 0.2}, ax=ax1)
ax1.plot(xnew2, ymu0, "--", color="b", alpha=0.7, label="Low contact mean")

# Plot highly-connected fit
az.plot_hdi(xnew2, ynew1, color="b", fill_kwargs={"alpha": 0.2}, ax=ax1)
ax1.plot(xnew2, ymu1,  color="b", alpha=0.7, label="High contact mean")

# Plot low-connected data
ax1.scatter(kdata.population[indx], T[indx],s=psize[indx],facecolors="none",edgecolors="k",alpha=0.8,lw=1,label="low contact")
# Plot high-connected data
ax1.scatter(kdata.population[~indx], T[~indx],s=psize[~indx],alpha=0.8,lw=1,label="high contact")

plt.setp(ax1.get_xticklabels(), ha="right", rotation=45)
ax1.set_xlabel("Population")
ax1.set_ylabel("Total tools")
ax1.set_xlim((-10000, 300000))
ax1.set_ylim(0,130)
ax1.legend(fontsize=8, ncol=2)
plt.tight_layout()
plt.savefig('sciencefits_nb.jpg',dpi=300);

# Poisson exposure rates

A key aspect of Poisson (or NB) mean parameters is that they can be thought of as rates - which means that they can be thought of as the count of events per unit exposure. Exposure can be any kind of standardization - per capita, per unit area, per kg, whatever - and to represent it, we can add a simple offset to our model, namely:


$$
\begin{align}
y_i \sim & Poisson(\lambda_i) \\
log(\lambda_i) = & log(\frac{\mu_i}{\tau_i}) = \beta_0 + \beta_1x_i
\end{align}
$$

is equivent to 

$$
\begin{align}
y_i \sim & Poisson(\mu_i) \\
log(\mu_i) = & log(\tau_i) + \beta_0 + \beta_1x_i
\end{align}
$$

where $\mu_i$ is the expected count and $\tau_i$ is the exposure. This often comes up so it is important to know about.